In [1]:
from trainer.glue_base import GlueAttentionApproxTrainer as Glue
import pickle

In [2]:
subsets = ["cola","mnli","mrpc","qnli","qqp","rte","sst2","stsb","wnli",]
kss = [0.1, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.999]

# subsets = ["cola","mrpc"]
# kss = [0.1, 0.25,]

def get_score(score):
    if 'accuracy' in score:
        return score['accuracy'], "acc"
    first_metric = list(score.keys())[0]
    return score[first_metric], first_metric

results = {}
i = 0
for subset in subsets:
    trainer = Glue(dataset=subset, factor=16, batch_size=-1, device=0)
    trainer.load()
    bert_score, metric_name = get_score(trainer.eval_base_model())
    scores = {'bert':bert_score}
    for ks in kss:
        sparse_score, _ = get_score(trainer.eval_sparse_model(ks=ks))
        scores[str(ks)] = sparse_score
        i += 1
        count = len(subsets) * len(kss)
        print(f'{i}/{count}')
    results[f"{subset} ({metric_name})"] = scores

with open('glue_benchmark.pkl', 'wb') as f:
    pickle.dump(results, f)

results

Trainer: cola


Reusing dataset glue (C:\Users\AinL\.cache\huggingface\datasets\glue\cola\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\cola\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-e8451e78068485b1.arrow
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\cola\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-d3ce574ceba0d772.arrow
Reusing dataset glue (C:\Users\AinL\.cache\huggingface\datasets\glue\cola\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\cola\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-64c999469ec3ef94.arrow
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\cola\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9d

metric score {'matthews_correlation': 0.5338774230813111}
avg occupy 0.34522852946633237


eval: 100%|██████████| 131/131 [00:07<00:00, 17.84it/s]


metric score {'matthews_correlation': -0.02674613601920812}
avg occupy 0.34522852946633237
1/72


...

metric score {'accuracy': 0.1975105253523705}
avg occupy 0.30243502260079336
32/72
Trainer: qqp


Reusing dataset glue (C:\Users\AinL\.cache\huggingface\datasets\glue\qqp\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\qqp\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-e7a33df65cd23373.arrow
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\qqp\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-ee58d2b2bd20f977.arrow
Reusing dataset glue (C:\Users\AinL\.cache\huggingface\datasets\glue\qqp\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\qqp\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-aa62b68e1840596a.arrow
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\qqp\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96

metric score {'accuracy': 0.9090774177590898, 'f1': 0.8781813361611877}
avg occupy 0.2734741473281857


eval:  20%|██        | 1016/5054 [00:52<03:27, 19.49it/s]


KeyboardInterrupt: 

In [ ]:
import pickle
import pandas as pd

with open('glue_benchmark.pkl', 'rb') as f:
    results = pickle.load(f)

data = []
subsets = list(results.keys())
factors = list(results[subsets[0]].keys())
for factor in factors:
    row = []
    for subset in subsets:
        row.append(results[subset][factor])
    data.append(row)
pd.DataFrame(data, columns=subsets, index=factors)

,cola (matthews_correlation),mnli (acc),mrpc (acc),qnli (acc),qqp (acc),rte (acc),sst2 (acc),stsb (pearson),wnli (acc)
bert,0.533877,0.842078,0.844058,0.915431,0.909077,0.725632,0.924312,0.880462,0.56338
0.1,-0.026746,0.336628,0.563478,0.479773,0.652090,0.454874,0.575688,0.024089,0.43662
0.25,0.280273,0.350280,0.655652,0.262493,0.796686,0.371841,0.596330,0.470021,0.43662
0.375,0.422845,0.354763,0.666667,0.205748,0.845362,0.361011,0.608945,0.573954,0.43662
0.5,0.464648,0.354661,0.668406,0.198975,0.857754,0.353791,0.628440,0.634881,0.43662
0.625,0.477545,0.355680,0.675362,0.197144,0.861860,0.353791,0.650229,0.678300,0.43662
0.75,0.491744,0.355374,0.677101,0.197511,0.862825,0.339350,0.653670,0.681150,0.43662
0.875,0.495515,0.355782,0.676522,0.197511,0.863319,0.346570,0.658257,0.677481,0.43662
